In [ ]:
def dynamic_learning_rate(progress_remaining):
    init_lr = 0.001
    return init_lr * progress_remaining

In [ ]:
# Experiment 1alt
LR = 0.0003
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

policy_kwargs = dict(
    net_arch = [256, 256, 256]
)

experiment_name = "a2c_experiment_1alt"


In [ ]:
# Experiment 1
LR = 0.0003
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

policy_kwargs = dict(
    net_arch = [64, 64]
)

experiment_name = "a2c_experiment_1"


In [ ]:
# Experiment 2
LR = 0.0005
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

policy_kwargs = dict(
    net_arch = [64, 64]
)

experiment_name = "a2c_experiment_2"

In [ ]:
# Experiment 3
LR = dynamic_learning_rate
GAMMA = 0.995
N_STEPS = 256
GAE_LAMBDA = 0.97

policy_kwargs = dict(
    net_arch = [64, 64]
)

experiment_name = "a2c_experiment_3"

In [ ]:
# Experiment 4
LR = 0.0003
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

policy_kwargs = dict(
    net_arch = [128, 128]
)

experiment_name = "a2c_experiment_4"

In [ ]:
# Experiment 5
LR = 0.0005
GAMMA = 0.99
N_STEPS = 256
GAE_LAMBDA = 0.95

policy_kwargs = dict(
    net_arch = [128, 128]
)

experiment_name = "a2c_experiment_5"

In [ ]:
# Experiment 6
LR = dynamic_learning_rate
GAMMA = 0.995
N_STEPS = 256
GAE_LAMBDA = 0.97

policy_kwargs = dict(
    net_arch = [128, 128]
)

experiment_name = "a2c_experiment_6"

In [ ]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np
import torch as th

# Create the CarRacing-v2 environment
env = gym.make('CarRacing-v2')
# env = Monitor(env)  # To log episode rewards

# Wrap the environment
# env = DummyVecEnv([lambda: env])

# Create the PPO model
model = A2C('MlpPolicy',
            env,
            verbose=1,
            n_steps=N_STEPS,
            learning_rate=LR,
            gamma=GAMMA,
            gae_lambda=GAE_LAMBDA,
            tensorboard_log="./ppo_car_racing_tensorboard/",
            device='auto',
            policy_kwargs=policy_kwargs
            )

# Train the model with the custom callback
model.learn(total_timesteps=50_000, tb_log_name=experiment_name, progress_bar=True, log_interval=1)

# Save the model
model.save(experiment_name)

# To use the model later, load it with:
# model = PPO.load("ppo_car_racing_v2")

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
import numpy as np

# load model

env = gym.make('CarRacing-v2', render_mode='human')
model = PPO.load("a2c_experiment_1")

obs, info = env.reset()
while True:
    try:
        action, _states = model.predict(obs)
        obs, rewards, dones, info, _ = env.step(action)
        env.render()
    except KeyboardInterrupt:
        break    